PKL reader

In [36]:
from Code.Utils.util_methods import NNUtils
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
import os
import pandas as pd
from rdkit import Chem
from rdkit.Chem import rdMolDescriptors
from tqdm import tqdm


cwd = os.getcwd()
base = f'{cwd}/../..'

## Give the path to the pkl file

In [38]:
FILEPATH = f'{base}/Dataset/Mass_spectra/cleaned_df.csv'
FILTER_COLUMNS = ['Name', 'Form', 'Mw', 'SMILES']
#FILTER_COLUMNS = None

## Extract the pkl file into a df

In [39]:
df = NNUtils.read_big_csv(FILEPATH)

Reading CSV file cleaned_df.csv: 183it [00:08, 22.26it/s]


In [40]:
if FILTER_COLUMNS:
    df = df[FILTER_COLUMNS]
print(df.shape)
df

(182516, 4)


,Name,Form,Mw,SMILES
0,Ethyl 3-ethoxy-cis-crotonate,C8H14O3,158,CCOC(/C)=C/C(=O)OCC
1,"Ethanone, 1-(3-methyloxiranyl)-",C5H8O2,100,CC1OC1C(C)=O
2,"2-Pentanone, 5-(1,2-propadienyloxy)-",C8H12O2,140,CC(=O)CCCO[CH]=[C]=[CH2]
3,"n-Butyl (1,1-dimethylbutyl) sulfide",C10H22S,174,CCCCSC(C)(C)CCC
4,"Sulfurous acid, isohexyl 2-propyl ester",C9H20O3S,208,CC(C)CCCO[S](=O)OC(C)C
...,...,...,...,...
182511,"Cyclopentanemethanamine, 2-amino-",C6H14N2,114,NCC1CCCC1N
182512,"1,6-Hexanediamine",C6H16N2,116,NCCCCCCN
182513,N-n-Butylpropionamide,C7H15NO,129,CCCCNC(=O)CC
182514,"Propanamide, N-hexyl-",C9H19NO,157,CCCCCCNC(=O)CC


In [41]:
df['smile_form_match'] = 99 # -2 means contains . and no match -1 means incorrect form 0 means no match 1 means match
df['new_form'] = df['Form']

c=0

for index in tqdm(range(df.shape[0])):

    # convert smiles to a molecule object
    if df.loc[index, 'SMILES'] is None:
        continue
    
    mol = Chem.MolFromSmiles(df.loc[index, 'SMILES'])
    if "." in df.loc[index, 'SMILES']:
        c+=1
    
    # check if the molecule is valid
    if mol:
        # calculate the molecular formula
        molecular_formula = rdMolDescriptors.CalcMolFormula(mol)
        if molecular_formula == df.loc[index, 'Form']:
            df.loc[index, 'smile_form_match'] = 1
        elif "." in df.loc[index, 'SMILES']:
            df.loc[index, 'smile_form_match'] = -2
        else:
            df.loc[index, 'smile_form_match'] = 0
            df.loc[index, 'new_form'] = molecular_formula
    else:
        df.loc[index, 'smile_form_match'] = -1
        
print(f'{c} molecules contain .')

 67%|██████▋   | 122771/182516 [00:24<00:10, 5640.51it/s][10:48:19] Conflicting single bond directions around double bond at index 4.
[10:48:19]   BondStereo set to STEREONONE and single bond directions set to NONE.
100%|██████████| 182516/182516 [00:34<00:00, 5265.55it/s]

0 molecules contain .


In [44]:
df[df['smile_form_match'] == -2]

,Name,Form,Mw,SMILES,smile_form_match,new_form


In [46]:
df[df['smile_form_match'] == 1]

,Name,Form,Mw,SMILES,smile_form_match,new_form
0,Ethyl 3-ethoxy-cis-crotonate,C8H14O3,158,CCOC(/C)=C/C(=O)OCC,1,C8H14O3
1,"Ethanone, 1-(3-methyloxiranyl)-",C5H8O2,100,CC1OC1C(C)=O,1,C5H8O2
2,"2-Pentanone, 5-(1,2-propadienyloxy)-",C8H12O2,140,CC(=O)CCCO[CH]=[C]=[CH2],1,C8H12O2
3,"n-Butyl (1,1-dimethylbutyl) sulfide",C10H22S,174,CCCCSC(C)(C)CCC,1,C10H22S
4,"Sulfurous acid, isohexyl 2-propyl ester",C9H20O3S,208,CC(C)CCCO[S](=O)OC(C)C,1,C9H20O3S
...,...,...,...,...,...,...
182511,"Cyclopentanemethanamine, 2-amino-",C6H14N2,114,NCC1CCCC1N,1,C6H14N2
182512,"1,6-Hexanediamine",C6H16N2,116,NCCCCCCN,1,C6H16N2
182513,N-n-Butylpropionamide,C7H15NO,129,CCCCNC(=O)CC,1,C7H15NO
182514,"Propanamide, N-hexyl-",C9H19NO,157,CCCCCCNC(=O)CC,1,C9H19NO
